In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from matplotlib.colors import Normalize
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("Maternal Health Risk Data Set.csv")
df.head()

#Aucune valeur manquante dans les donnees
df.isnull().values.sum()

In [ ]:
# Répartition graphique des patientes sur un tableau récapitulatif des premières variables
plt.figure(figsize=(25,5))
plt.subplot(1,4,1)
sns.boxplot(x='RiskLevel',y='Age',data=df)
plt.subplot(1,4,2)
sns.boxplot(x='RiskLevel',y='SystolicBP',data=df)
plt.subplot(1,4,3)
sns.boxplot(x='RiskLevel',y='DiastolicBP',data=df)
plt.subplot(1,4,4)
sns.boxplot(x='RiskLevel',y='BS',data=df)


In [ ]:
values, bins, hist = plt.hist(df['RiskLevel'],edgecolor='red',bins=3)



plt.xlabel('Risk class level')
plt.ylabel('Number of patients')
plt.title('Graphique du nombre de patientes par rapport à la classe de risque de grossesse')


In [ ]:
markers = {"low risk": "s", "mid risk": "X","high risk":"o"}
sns.pairplot(df, hue='RiskLevel', height=3,markers=markers)

In [ ]:
sns.set_style('whitegrid')
sns.FacetGrid(df, hue='RiskLevel', height=3).map(plt.scatter, 'BS', 'SystolicBP').add_legend()
sns.FacetGrid(df, hue='RiskLevel', height=3).map(plt.scatter, 'SystolicBP', 'DiastolicBP').add_legend()
sns.FacetGrid(df, hue='RiskLevel', height=3).map(plt.scatter, 'DiastolicBP', 'SystolicBP').add_legend()
plt.show()

In [ ]:
sns.heatmap(df.corr(),annot=True, vmin=-1, vmax=1, cmap=sns.diverging_palette(20, 220, as_cmap=True))

Méthode Knn

In [ ]:
#On se place dans le cas de la stratégie 1 comme aux premiers exercices du TP1
X = df.drop("RiskLevel", axis=1)
Y = df['RiskLevel']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
#Normalisation des jeux de donnees comme indique en Partie 1 du TP1
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.fit_transform(X_test)

In [ ]:
#Application de l’algorithme des k plus proches voisins, pour chaque valeur de p_k, de sorte à pouvoir évaluer l’erreur de
#classification en apprentissage et en validation
range_k = ( 1, 5, 10, 15, 20, 25 )
f1 = []
accuracy = []

for i in range_k:
    neigh  = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X_train_scale, np.ravel(Y_train))
    y_pred = neigh.predict(X_test_scale)

#Fonction f1-score paramètre macro prend la moyenne des F1-score par classe, tiens compte taille echantillon
    print("##########################################################")
    print('k=',i)
    print(classification_report(Y_test,y_pred,digits=3))
    print("##########################################################")
    accuracy.append(accuracy_score(Y_test,y_pred))
    f1.append(f1_score(Y_test,y_pred, average='macro'))

plt.plot(range_k,accuracy)
plt.xlabel('Valeur K testé')
plt.ylabel('Accuracy')
plt.title("Les valeurs p_k par rapport à l'accuracy")
plt.show()

plt.plot(range_k,f1)
plt.xlabel('Valeur K testé')
plt.ylabel('f1_score')
plt.title('Les valeurs p_k par rapport au f1_score')
plt.show()




    


In [ ]:
plot_confusion_matrix(neigh,X_test_scale, Y_test)


Méthode SVM

In [ ]:


#appliquer GridSearchCV pour chercher les meilleurs parametres
param_grid = [
    {'kernel': ['rbf'], 'gamma': np.arange(0,7,0.1), 'C': np.arange(0,7,0.1)},
    {'kernel': ['poly'], 'degree': [1, 2, 3,4], 'C': np.arange(0,7,0.1)}
]

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X, Y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)
###########La cellule met 19min environ à s'exécuter
##########Voici le resultat obtenu:
##########The best parameters are {'C': 4.4, 'gamma':6.9, 'kernel': 'rbf'} with a score of 0.82

In [ ]:
#Création d'un classifieur avec paramètre C et gamma
svc=SVC(C=4.4,gamma=6.9) 
svc.fit(X_train_scale,Y_train)

#Prédire sur l'ensemble de test
y_pred=svc.predict(X_test_scale)
print('Model accuracy score with rbf kernel and C=4.4 and gamma=6.9: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))

In [ ]:
plot_confusion_matrix(svc, X_test_scale, Y_test)  